In [1]:
def get_vectorizer(vocab_file):
    print(vocab_file)
    corpus = None
    with open(vocab_file, encoding='UTF-8') as f:
        corpus = f.readlines() 
    corpus = [w.strip() for w in corpus]
    print(len(corpus))
    voca = dict()
    for idx, term in enumerate(corpus):
        voca[term] = idx

    vectorizer = CountVectorizer(vocabulary=voca)
    return vectorizer

In [44]:
def get_vectorizer_new(vocab_file):
    print(vocab_file)
    corpus = None
    with open(vocab_file, encoding='UTF-8') as f:
        corpus = f.readlines() 
    corpus = [w.strip() for w in corpus]
    print(len(corpus))
    voca = dict()
    for idx, term in enumerate(corpus):
        voca[term] = idx

    vectorizer = CountVectorizer(vocabulary=voca, ngram_range=(1, 10))
    return vectorizer

In [3]:
def vars(a, axis=None):
    """ Variance of sparse matrix a
    var = mean(a**2) - mean(a)**2
    """
    a_squared = a.copy()
    a_squared.data **= 2
    return a_squared.mean(axis) - np.square(a.mean(axis))

In [4]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [7]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [8]:
import re 

In [3]:
# dirc = r'D:\studying\UIUC courses\STAT542\project\Project3\{}'

# file = dirc.format('alldata.tsv')

# sentiment = []
# review = []
# cleaner = re.compile('<.*?>') 

# with open(file, encoding='UTF-8') as f:
#     lines = f.readlines()
#     for i in range(1, len(lines)):
#         uid, senti, sc, rev = lines[i].split('\t')
#         sentiment.append(int(senti))
#         rev = re.sub(cleaner, '', rev)
#         rev = rev.strip().strip('"')
#         review.append(re.sub(cleaner, '', rev))

# alldata = pd.DataFrame({'sentiment': sentiment, 'review': review})

# del sentiment, review

In [9]:
cleaner = re.compile('<.*?>') 

In [10]:
all_file = r'D:\studying\UIUC courses\STAT542\project\Project3\splits\alldata.tsv'

alldata = pd.read_csv(all_file, sep='\t', encoding='utf-8')

alldata['review'] = alldata['review'].map(lambda s: re.sub(cleaner, '', s))

In [11]:
Y = alldata['sentiment']

In [12]:
stop_words = ["i", "me", "my", "myself", 
               "we", "our", "ours", "ourselves", 
               "you", "your", "yours", 
               "their", "they", "his", "her", 
               "she", "he", "a", "an", "and",
               "is", "was", "are", "were", 
               "him", "himself", "has", "have", 
               "it", "its", "the", "us"]

In [14]:
alldata.shape  

(50000, 4)

In [15]:
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 8), min_df=0.001, max_df=0.5)
corpus = alldata['review'].values
X = vectorizer.fit_transform(corpus)

In [16]:
X.shape

(50000, 31393)

In [17]:
indi = (alldata['sentiment'] == 1)

In [18]:
size = X.shape[1]
summ = np.zeros((size, 4))
# mean_1
summ[:,0] = np.array(X[indi].mean(axis=0)).reshape(-1)
# var_1 
summ[:,1] = np.array(vars(X[indi], axis=0)).reshape(-1)

# mean_2
summ[:,2] = np.array(X[~indi].mean(axis=0)).reshape(-1)
# var_2
summ[:,3] = np.array(vars(X[~indi], axis=0)).reshape(-1)

In [19]:
summ 

array([[0.00412   , 0.00690303, 0.0042    , 0.00482236],
       [0.01092   , 0.01984075, 0.01384   , 0.02348845],
       [0.001     , 0.001559  , 0.002     , 0.002716  ],
       ...,
       [0.00696   , 0.01515156, 0.02596   , 0.09200608],
       [0.0058    , 0.00744636, 0.0054    , 0.00657084],
       [0.00084   , 0.00115929, 0.00204   , 0.00299584]])

In [20]:
n1 = sum(alldata['sentiment'])
n2 = len(alldata) - n1

tstat = (summ[:,0] - summ[:,2]) / np.sqrt(summ[:,1] / n1 + summ[:,3] / n2)

In [33]:
idx = np.argsort(-np.abs(tstat))[:2500]

In [34]:
pos_idx = idx[tstat[idx] >= 0]
neg_idx = idx[tstat[idx] < 0]

In [35]:
words = np.array(vectorizer.get_feature_names(), dtype='str')

In [36]:
pos_words = words[pos_idx]
neg_words = words[neg_idx]

In [37]:
len(pos_words)

910

In [38]:
len(neg_words)

1590

In [40]:
neg_words[-10:]

array(['but worst', 'thinking that', 'up', 'comes off', 'plain stupid',
       'so poor', 'makers of this', 'but failed', 'does not make',
       'had to'], dtype='<U32')

In [41]:
F = r'D:\studying\UIUC courses\STAT542\project\Project3\splits\myvocab_10_2500.txt'

In [42]:
with open(F, mode='w', encoding='UTF-8') as f:
    for w in pos_words:
        f.write(w + '\n')
    for w in neg_words:
        f.write(w + '\n')  

# vocab txt

In [54]:
vocab_file = r'D:\studying\UIUC courses\STAT542\project\Project3\splits\myvocab.txt'

In [14]:
# file = dirc.format('alldata.tsv')

# sentiment = []
# review = []
# cleaner = re.compile('<.*?>')
# with open(file, encoding='UTF-8') as f:
#     lines = f.readlines()
#     for i in range(1, len(lines)):
#         uid, senti, sc, rev = lines[i].split('\t')
#         sentiment.append(int(senti))
#         rev = re.sub(cleaner, '', rev)
#         rev = rev.strip().strip('"')
#         review.append(re.sub(cleaner, '', rev))

# alldata = pd.DataFrame({'sentiment': sentiment, 'review': review})

# del sentiment, review

In [55]:
corpus = None
with open(vocab_file, encoding='UTF-8') as f:
    corpus = f.readlines() 

corpus = [w.strip() for w in corpus]

In [165]:
corpus[-1]

'be fair'

In [57]:
voca = dict()
for idx, term in enumerate(corpus):
    voca[term] = idx

vectorizer = CountVectorizer(vocabulary=voca)

In [ ]:
vectorizer.get_feature_names()

In [58]:
len(vectorizer.get_feature_names())

2000

In [16]:
text2idx = vectorizer.vocabulary_

In [17]:
X = vectorizer.transform(alldata['review'].values).toarray()

X_train = pd.DataFrame(X, columns=vectorizer.get_feature_names())

Y_train = alldata['sentiment']

# get 1000 vocab lasso

In [45]:
all_file = r'D:\studying\UIUC courses\STAT542\project\Project3\splits\alldata.tsv'
train = pd.read_csv(all_file, sep='\t', encoding='utf-8')
train['review'] = train['review'].map(lambda s: re.sub(cleaner, '', s))

In [46]:
vocab_file = r'D:\studying\UIUC courses\STAT542\project\Project3\splits\myvocab_10_2500.txt'

vectorizer = get_vectorizer_new(vocab_file)

D:\studying\UIUC courses\STAT542\project\Project3\splits\myvocab_10_2500.txt
2500


In [48]:
X = vectorizer.transform(train['review'].values).toarray()
X_train = pd.DataFrame(X, columns=vectorizer.get_feature_names())
Y_train = train['sentiment']

In [49]:
X_train.shape

(50000, 2500)

In [52]:
X_train['one of greatest'].sum() 

1

In [55]:
for c in (0.095, 0.0955, 0.096, 0.0965, 0.097):
    lasso = LogisticRegression(penalty='l1', C=c, solver='liblinear', random_state=2021)
    lasso.fit(X_train, Y_train)
    v = np.sum(lasso.coef_.reshape(-1) != 0)
    print(c, v)

0.095 995
0.0955 999
0.096 1002
0.0965 1002
0.097 1004


In [56]:
lasso = LogisticRegression(penalty='l1', C=0.0955, solver='liblinear', random_state=2021)

In [57]:
lasso.fit(X_train, Y_train)

LogisticRegression(C=0.0955, penalty='l1', random_state=2021,
                   solver='liblinear')

In [58]:
sum(lasso.coef_.reshape(-1) != 0)

999

In [59]:
coef = lasso.coef_.reshape(-1)

In [60]:
features = np.array(vectorizer.get_feature_names())

In [61]:
len(features[coef != 0].tolist())

999

In [62]:
vocab_file = 'D:\\studying\\UIUC courses\\STAT542\\project\\Project3\\splits\\vocab_lasso_10_999.txt'

In [63]:
with open(vocab_file, mode='w', encoding='UTF-8') as f:
    for w in features[coef != 0].tolist():
        f.write(w + '\n')

# alldata get 1000 vocab

In [ ]:
all_file

In [134]:
train = pd.read_csv(all_file, sep='\t', encoding='utf-8')
train['review'] = train['review'].map(lambda s: re.sub(cleaner, '', s))

In [135]:
X = vectorizer.transform(train['review'].values).toarray()
X_train = pd.DataFrame(X, columns=vectorizer.get_feature_names())
Y_train = train['sentiment']

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.15, random_state=2021)

In [150]:
params = {'objective':'binary:logistic', 'n_estimators':1000, 'max_depth':10, 
         'learning_rate':0.1, 'n_jobs':-1, 'subsample':0.7, 'random_state':2021,
         'use_label_encoder':False}

clf = xgb.XGBClassifier(**params)

In [151]:
clf.fit(X_train, y_train, eval_metric='auc', eval_set=[(X_test, y_test)], early_stopping_rounds=30)

[0]	validation_0-auc:0.77665
[1]	validation_0-auc:0.80039
[2]	validation_0-auc:0.80548
[3]	validation_0-auc:0.81450
[4]	validation_0-auc:0.81739
[5]	validation_0-auc:0.82265
[6]	validation_0-auc:0.82783
[7]	validation_0-auc:0.83138
[8]	validation_0-auc:0.83512
[9]	validation_0-auc:0.83905
[10]	validation_0-auc:0.84271
[11]	validation_0-auc:0.84767
[12]	validation_0-auc:0.84991
[13]	validation_0-auc:0.85289
[14]	validation_0-auc:0.85588
[15]	validation_0-auc:0.85877
[16]	validation_0-auc:0.86172
[17]	validation_0-auc:0.86411
[18]	validation_0-auc:0.86586
[19]	validation_0-auc:0.86913
[20]	validation_0-auc:0.87164
[21]	validation_0-auc:0.87413
[22]	validation_0-auc:0.87653
[23]	validation_0-auc:0.87832
[24]	validation_0-auc:0.88014
[25]	validation_0-auc:0.88187
[26]	validation_0-auc:0.88315
[27]	validation_0-auc:0.88474
[28]	validation_0-auc:0.88625
[29]	validation_0-auc:0.88756
[30]	validation_0-auc:0.88897
[31]	validation_0-auc:0.89023
[32]	validation_0-auc:0.89160
[33]	validation_0-au

[268]	validation_0-auc:0.94319
[269]	validation_0-auc:0.94327
[270]	validation_0-auc:0.94331
[271]	validation_0-auc:0.94333
[272]	validation_0-auc:0.94323
[273]	validation_0-auc:0.94329
[274]	validation_0-auc:0.94337
[275]	validation_0-auc:0.94340
[276]	validation_0-auc:0.94347
[277]	validation_0-auc:0.94340
[278]	validation_0-auc:0.94337
[279]	validation_0-auc:0.94337
[280]	validation_0-auc:0.94332
[281]	validation_0-auc:0.94340
[282]	validation_0-auc:0.94348
[283]	validation_0-auc:0.94351
[284]	validation_0-auc:0.94356
[285]	validation_0-auc:0.94361
[286]	validation_0-auc:0.94363
[287]	validation_0-auc:0.94366
[288]	validation_0-auc:0.94372
[289]	validation_0-auc:0.94378
[290]	validation_0-auc:0.94378
[291]	validation_0-auc:0.94387
[292]	validation_0-auc:0.94391
[293]	validation_0-auc:0.94388
[294]	validation_0-auc:0.94385
[295]	validation_0-auc:0.94390
[296]	validation_0-auc:0.94393
[297]	validation_0-auc:0.94397
[298]	validation_0-auc:0.94398
[299]	validation_0-auc:0.94401
[300]	va

[533]	validation_0-auc:0.94810
[534]	validation_0-auc:0.94811
[535]	validation_0-auc:0.94818
[536]	validation_0-auc:0.94820
[537]	validation_0-auc:0.94822
[538]	validation_0-auc:0.94824
[539]	validation_0-auc:0.94828
[540]	validation_0-auc:0.94832
[541]	validation_0-auc:0.94836
[542]	validation_0-auc:0.94837
[543]	validation_0-auc:0.94841
[544]	validation_0-auc:0.94842
[545]	validation_0-auc:0.94845
[546]	validation_0-auc:0.94845
[547]	validation_0-auc:0.94849
[548]	validation_0-auc:0.94851
[549]	validation_0-auc:0.94853
[550]	validation_0-auc:0.94851
[551]	validation_0-auc:0.94854
[552]	validation_0-auc:0.94853
[553]	validation_0-auc:0.94853
[554]	validation_0-auc:0.94851
[555]	validation_0-auc:0.94853
[556]	validation_0-auc:0.94856
[557]	validation_0-auc:0.94857
[558]	validation_0-auc:0.94857
[559]	validation_0-auc:0.94860
[560]	validation_0-auc:0.94864
[561]	validation_0-auc:0.94866
[562]	validation_0-auc:0.94870
[563]	validation_0-auc:0.94868
[564]	validation_0-auc:0.94871
[565]	va

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1,
              random_state=2021, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.7, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [ ]:
'[741]	validation_0-auc:0.94974'

In [155]:
impo = clf.feature_importances_

feature_idx = np.argsort(-impo)[:1000]

features = np.array(vectorizer.get_feature_names())


fea_1000 = features[feature_idx]

In [158]:
fea_1000_lst = fea_1000.tolist() 

In [162]:
vocab_file = 'D:\\studying\\UIUC courses\\STAT542\\project\\Project3\\splits\\vocab_1000.txt'

In [163]:
with open(vocab_file, mode='w', encoding='UTF-8') as f:
    for w in fea_1000_lst:
        f.write(w + '\n')

# one split

In [92]:
split_dirc = 'D:\\studying\\UIUC courses\\STAT542\\project\\Project3\\splits\\split_{}\\'
i = 1
dirc = split_dirc.format(i)

train_file = dirc + 'train.tsv'
test_file = dirc + 'test.tsv'
test_y_file = dirc + 'test_y.tsv'

In [93]:
train_file

'D:\\studying\\UIUC courses\\STAT542\\project\\Project3\\splits\\split_1\\train.tsv'

In [94]:
vocab_file = r'D:\studying\UIUC courses\STAT542\project\Project3\splits\vocab_lasso_10_999.txt'
# vocab_file = 'D:\\studying\\UIUC courses\\STAT542\\project\\Project3\\splits\\vocab_lasso_840.txt'
vectorizer = get_vectorizer_new(vocab_file)

D:\studying\UIUC courses\STAT542\project\Project3\splits\vocab_lasso_10_999.txt
999


In [95]:
# vocab_file = 'D:\\studying\\UIUC courses\\STAT542\\project\\Project3\\splits\\vocab_lasso_840.txt'
# vectorizer_new = get_vectorizer_new(vocab_file)

In [96]:
len(vectorizer.get_feature_names())

999

In [97]:
cleaner = re.compile('<.*?>') 

In [338]:
# X = vectorizer.transform(train['review'].values).toarray()
# X_new = vectorizer_new.transform(train['review'].values).toarray()

# np.sum(X != X_new)

In [98]:
train = pd.read_csv(train_file, sep='\t', encoding='utf-8')
train['review'] = train['review'].map(lambda s: re.sub(cleaner, '', s))

test = pd.read_csv(test_file, sep='\t', encoding='utf-8')
test['review'] = test['review'].map(lambda s: re.sub(cleaner, '', s))

test_y = pd.read_csv(test_y_file, sep='\t', encoding='utf-8')
#test_y['sentiment']

In [ ]:
#(X_train != 0).sum(axis=1).sort_values()

In [99]:
X = vectorizer.transform(train['review'].values).toarray()
X_train = pd.DataFrame(X, columns=vectorizer.get_feature_names())
Y_train = train['sentiment']

In [100]:
X = vectorizer.transform(test['review'].values).toarray()
X_test = pd.DataFrame(X, columns=vectorizer.get_feature_names())

In [ ]:
X_train.sum()

In [101]:
np.sum(X_test['supposed to be'] != 0)

733

In [185]:
# X_train[X_train > 0] = 1

# X_test[X_test > 0] = 1

In [104]:
params = {'objective':'binary:logistic', 'n_estimators':4000, 'max_depth':5, 
         'learning_rate':0.05, 'n_jobs':-1, 'subsample':0.5, 'reg_lambda':1,  
          'random_state':2021, 'use_label_encoder':False}

clf = xgb.XGBClassifier(**params)

In [105]:
X_train.shape 

(25000, 999)

In [106]:
clf.fit(X_train, Y_train, eval_metric=['auc'], eval_set=[(X_train, Y_train), (X_test, test_y['sentiment'])], 
       early_stopping_rounds=150)

[0]	validation_0-auc:0.74360	validation_1-auc:0.73910
[1]	validation_0-auc:0.74819	validation_1-auc:0.74324
[2]	validation_0-auc:0.75065	validation_1-auc:0.74618
[3]	validation_0-auc:0.75872	validation_1-auc:0.75485
[4]	validation_0-auc:0.77735	validation_1-auc:0.77146
[5]	validation_0-auc:0.77842	validation_1-auc:0.77234
[6]	validation_0-auc:0.77770	validation_1-auc:0.77187
[7]	validation_0-auc:0.78567	validation_1-auc:0.77983
[8]	validation_0-auc:0.79167	validation_1-auc:0.78669
[9]	validation_0-auc:0.79263	validation_1-auc:0.78799
[10]	validation_0-auc:0.79763	validation_1-auc:0.79262
[11]	validation_0-auc:0.80394	validation_1-auc:0.79915
[12]	validation_0-auc:0.80552	validation_1-auc:0.80059
[13]	validation_0-auc:0.81131	validation_1-auc:0.80591
[14]	validation_0-auc:0.81233	validation_1-auc:0.80697
[15]	validation_0-auc:0.81555	validation_1-auc:0.80997
[16]	validation_0-auc:0.81690	validation_1-auc:0.81138
[17]	validation_0-auc:0.81836	validation_1-auc:0.81282
[18]	validation_0-au

[149]	validation_0-auc:0.93040	validation_1-auc:0.91029
[150]	validation_0-auc:0.93057	validation_1-auc:0.91043
[151]	validation_0-auc:0.93096	validation_1-auc:0.91079
[152]	validation_0-auc:0.93133	validation_1-auc:0.91119
[153]	validation_0-auc:0.93145	validation_1-auc:0.91131
[154]	validation_0-auc:0.93168	validation_1-auc:0.91153
[155]	validation_0-auc:0.93191	validation_1-auc:0.91168
[156]	validation_0-auc:0.93223	validation_1-auc:0.91195
[157]	validation_0-auc:0.93255	validation_1-auc:0.91224
[158]	validation_0-auc:0.93281	validation_1-auc:0.91240
[159]	validation_0-auc:0.93312	validation_1-auc:0.91261
[160]	validation_0-auc:0.93337	validation_1-auc:0.91288
[161]	validation_0-auc:0.93374	validation_1-auc:0.91306
[162]	validation_0-auc:0.93407	validation_1-auc:0.91332
[163]	validation_0-auc:0.93435	validation_1-auc:0.91347
[164]	validation_0-auc:0.93457	validation_1-auc:0.91370
[165]	validation_0-auc:0.93490	validation_1-auc:0.91397
[166]	validation_0-auc:0.93503	validation_1-auc:

[296]	validation_0-auc:0.95642	validation_1-auc:0.93115
[297]	validation_0-auc:0.95653	validation_1-auc:0.93121
[298]	validation_0-auc:0.95663	validation_1-auc:0.93123
[299]	validation_0-auc:0.95671	validation_1-auc:0.93128
[300]	validation_0-auc:0.95683	validation_1-auc:0.93135
[301]	validation_0-auc:0.95694	validation_1-auc:0.93139
[302]	validation_0-auc:0.95700	validation_1-auc:0.93147
[303]	validation_0-auc:0.95710	validation_1-auc:0.93155
[304]	validation_0-auc:0.95722	validation_1-auc:0.93162
[305]	validation_0-auc:0.95731	validation_1-auc:0.93166
[306]	validation_0-auc:0.95742	validation_1-auc:0.93173
[307]	validation_0-auc:0.95751	validation_1-auc:0.93178
[308]	validation_0-auc:0.95759	validation_1-auc:0.93187
[309]	validation_0-auc:0.95766	validation_1-auc:0.93191
[310]	validation_0-auc:0.95777	validation_1-auc:0.93203
[311]	validation_0-auc:0.95782	validation_1-auc:0.93210
[312]	validation_0-auc:0.95792	validation_1-auc:0.93216
[313]	validation_0-auc:0.95803	validation_1-auc:

[443]	validation_0-auc:0.96778	validation_1-auc:0.93987
[444]	validation_0-auc:0.96783	validation_1-auc:0.93993
[445]	validation_0-auc:0.96792	validation_1-auc:0.93997
[446]	validation_0-auc:0.96797	validation_1-auc:0.94001
[447]	validation_0-auc:0.96802	validation_1-auc:0.94005
[448]	validation_0-auc:0.96805	validation_1-auc:0.94008
[449]	validation_0-auc:0.96809	validation_1-auc:0.94013
[450]	validation_0-auc:0.96814	validation_1-auc:0.94015
[451]	validation_0-auc:0.96821	validation_1-auc:0.94021
[452]	validation_0-auc:0.96826	validation_1-auc:0.94027
[453]	validation_0-auc:0.96829	validation_1-auc:0.94027
[454]	validation_0-auc:0.96834	validation_1-auc:0.94031
[455]	validation_0-auc:0.96839	validation_1-auc:0.94032
[456]	validation_0-auc:0.96842	validation_1-auc:0.94037
[457]	validation_0-auc:0.96847	validation_1-auc:0.94042
[458]	validation_0-auc:0.96853	validation_1-auc:0.94046
[459]	validation_0-auc:0.96857	validation_1-auc:0.94050
[460]	validation_0-auc:0.96864	validation_1-auc:

[590]	validation_0-auc:0.97435	validation_1-auc:0.94491
[591]	validation_0-auc:0.97438	validation_1-auc:0.94496
[592]	validation_0-auc:0.97440	validation_1-auc:0.94499
[593]	validation_0-auc:0.97443	validation_1-auc:0.94499
[594]	validation_0-auc:0.97449	validation_1-auc:0.94502
[595]	validation_0-auc:0.97453	validation_1-auc:0.94501
[596]	validation_0-auc:0.97455	validation_1-auc:0.94505
[597]	validation_0-auc:0.97459	validation_1-auc:0.94507
[598]	validation_0-auc:0.97462	validation_1-auc:0.94506
[599]	validation_0-auc:0.97466	validation_1-auc:0.94508
[600]	validation_0-auc:0.97470	validation_1-auc:0.94509
[601]	validation_0-auc:0.97473	validation_1-auc:0.94510
[602]	validation_0-auc:0.97480	validation_1-auc:0.94513
[603]	validation_0-auc:0.97483	validation_1-auc:0.94517
[604]	validation_0-auc:0.97486	validation_1-auc:0.94518
[605]	validation_0-auc:0.97490	validation_1-auc:0.94518
[606]	validation_0-auc:0.97494	validation_1-auc:0.94521
[607]	validation_0-auc:0.97499	validation_1-auc:

...

[2182]	validation_0-auc:0.99396	validation_1-auc:0.95739
[2183]	validation_0-auc:0.99397	validation_1-auc:0.95740
[2184]	validation_0-auc:0.99397	validation_1-auc:0.95740
[2185]	validation_0-auc:0.99398	validation_1-auc:0.95740
[2186]	validation_0-auc:0.99398	validation_1-auc:0.95742
[2187]	validation_0-auc:0.99399	validation_1-auc:0.95742
[2188]	validation_0-auc:0.99399	validation_1-auc:0.95743
[2189]	validation_0-auc:0.99401	validation_1-auc:0.95744
[2190]	validation_0-auc:0.99403	validation_1-auc:0.95743
[2191]	validation_0-auc:0.99403	validation_1-auc:0.95742
[2192]	validation_0-auc:0.99404	validation_1-auc:0.95742
[2193]	validation_0-auc:0.99404	validation_1-auc:0.95742
[2194]	validation_0-auc:0.99404	validation_1-auc:0.95742
[2195]	validation_0-auc:0.99405	validation_1-auc:0.95743
[2196]	validation_0-auc:0.99405	validation_1-auc:0.95743
[2197]	validation_0-auc:0.99405	validation_1-auc:0.95744
[2198]	validation_0-auc:0.99405	validation_1-auc:0.95743
[2199]	validation_0-auc:0.99406

[2326]	validation_0-auc:0.99465	validation_1-auc:0.95753
[2327]	validation_0-auc:0.99466	validation_1-auc:0.95754
[2328]	validation_0-auc:0.99466	validation_1-auc:0.95753
[2329]	validation_0-auc:0.99467	validation_1-auc:0.95754
[2330]	validation_0-auc:0.99467	validation_1-auc:0.95754
[2331]	validation_0-auc:0.99467	validation_1-auc:0.95753
[2332]	validation_0-auc:0.99468	validation_1-auc:0.95753
[2333]	validation_0-auc:0.99469	validation_1-auc:0.95753
[2334]	validation_0-auc:0.99469	validation_1-auc:0.95753
[2335]	validation_0-auc:0.99469	validation_1-auc:0.95753
[2336]	validation_0-auc:0.99470	validation_1-auc:0.95754
[2337]	validation_0-auc:0.99471	validation_1-auc:0.95754
[2338]	validation_0-auc:0.99471	validation_1-auc:0.95754
[2339]	validation_0-auc:0.99472	validation_1-auc:0.95754
[2340]	validation_0-auc:0.99473	validation_1-auc:0.95753
[2341]	validation_0-auc:0.99473	validation_1-auc:0.95753
[2342]	validation_0-auc:0.99474	validation_1-auc:0.95753
[2343]	validation_0-auc:0.99474

[2470]	validation_0-auc:0.99528	validation_1-auc:0.95761
[2471]	validation_0-auc:0.99528	validation_1-auc:0.95761
[2472]	validation_0-auc:0.99528	validation_1-auc:0.95761
[2473]	validation_0-auc:0.99529	validation_1-auc:0.95761
[2474]	validation_0-auc:0.99530	validation_1-auc:0.95762
[2475]	validation_0-auc:0.99530	validation_1-auc:0.95762
[2476]	validation_0-auc:0.99530	validation_1-auc:0.95762
[2477]	validation_0-auc:0.99531	validation_1-auc:0.95762
[2478]	validation_0-auc:0.99531	validation_1-auc:0.95763
[2479]	validation_0-auc:0.99531	validation_1-auc:0.95763
[2480]	validation_0-auc:0.99532	validation_1-auc:0.95763
[2481]	validation_0-auc:0.99532	validation_1-auc:0.95764
[2482]	validation_0-auc:0.99532	validation_1-auc:0.95764
[2483]	validation_0-auc:0.99532	validation_1-auc:0.95764
[2484]	validation_0-auc:0.99533	validation_1-auc:0.95764
[2485]	validation_0-auc:0.99534	validation_1-auc:0.95764
[2486]	validation_0-auc:0.99535	validation_1-auc:0.95764
[2487]	validation_0-auc:0.99536

[2614]	validation_0-auc:0.99587	validation_1-auc:0.95771
[2615]	validation_0-auc:0.99588	validation_1-auc:0.95771
[2616]	validation_0-auc:0.99588	validation_1-auc:0.95770
[2617]	validation_0-auc:0.99588	validation_1-auc:0.95770
[2618]	validation_0-auc:0.99589	validation_1-auc:0.95770
[2619]	validation_0-auc:0.99589	validation_1-auc:0.95770
[2620]	validation_0-auc:0.99589	validation_1-auc:0.95770
[2621]	validation_0-auc:0.99590	validation_1-auc:0.95771
[2622]	validation_0-auc:0.99590	validation_1-auc:0.95770
[2623]	validation_0-auc:0.99591	validation_1-auc:0.95771
[2624]	validation_0-auc:0.99590	validation_1-auc:0.95771
[2625]	validation_0-auc:0.99591	validation_1-auc:0.95771
[2626]	validation_0-auc:0.99592	validation_1-auc:0.95771
[2627]	validation_0-auc:0.99593	validation_1-auc:0.95771
[2628]	validation_0-auc:0.99594	validation_1-auc:0.95771
[2629]	validation_0-auc:0.99594	validation_1-auc:0.95771
[2630]	validation_0-auc:0.99594	validation_1-auc:0.95771
[2631]	validation_0-auc:0.99594

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=4000, n_jobs=-1, num_parallel_tree=1,
              random_state=2021, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.5, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [107]:
len(clf.feature_importances_)

999

In [108]:
accuracy_score(test_y['sentiment'].values, clf.predict(X_test))

0.89104

In [109]:
true_y = test_y['sentiment'].values

In [111]:
for c in np.linspace(0.1,0.5,10):
    ridge = LogisticRegression(C=c, random_state=2021, max_iter=1000)
    ridge.fit(X_train, Y_train)
    pred_test = ridge.predict_proba(X_test)
    auc = roc_auc_score(true_y, pred_test[:,1])
    print(c, auc)

0.1 0.9583255271635795
0.14444444444444446 0.9586735479873055
0.18888888888888888 0.958818963065754
0.23333333333333334 0.9588804097524397
0.2777777777777778 0.9589051522678784
0.32222222222222224 0.9589139011086971
0.3666666666666667 0.9589096258887504
0.4111111111111111 0.9588969218294785
0.4555555555555556 0.9588788929453629
0.5 0.9588617088651888


In [89]:
ridge = LogisticRegression(C=1, random_state=2021, max_iter=1000)
ridge.fit(X_train, Y_train)

LogisticRegression(C=1, max_iter=1000, random_state=2021)

In [90]:
pred_test = ridge.predict_proba(X_test)

roc_auc_score(test_y['sentiment'].values, pred_test[:,1])

In [263]:
accuracy_score(test_y['sentiment'].values, ridge.predict(X_test))

0.88432

In [108]:
pred_train = clf.predict_proba(X_train)

In [109]:
pred_train[:,0]

array([0.2547664 , 0.3080622 , 0.6953828 , ..., 0.96667534, 0.64882183,
       0.54413795], dtype=float32)

In [29]:
pred_test = clf.predict_proba(X_test)

In [30]:
roc_auc_score(test_y['sentiment'].values, pred_test[:,1])

0.9423242973082415

In [118]:
clf.feature_importances_

array([0.01319648, 0.01110363, 0.01131603, ..., 0.        , 0.        ,
       0.        ], dtype=float32)

In [120]:
len(clf.feature_importances_)

2000